In [1]:
import pandas as pd
import pyslim
import tskit
import numpy as np

# Parallelizing SLiM simulations in a phylogenetic tree

In [2]:
path_to_tsv = "phylo.tsv"
path_to_make = "parallel_sims.make"
path_to_slimscript = "simulate_branch.slim"
path_to_unioned = "unioned.trees"

## Simulating the branches

### Creating the Makefile to run simulations

In the paper, we just wrote down the Makefile.
For a more complicated phylogeny we might want to generate it programatically.
Here's a way to do it, using the data in this tsv file:

In [3]:
! cat phylo.tsv

child	parent	popsize	edgelen
root		500	2000
C	root	50	250
I	root	100	200
B	I	70	50
A	I	40	50


In [4]:
# Reading the phylogeny data frame
df = pd.read_csv(path_to_tsv, sep="\t")
df = df.fillna('')

## Creating intermediate tree sequences filenames
df["infile"] = df.parent + ".trees"
df["outfile"] = df.child + ".trees"
df.loc[df["infile"]==".trees", "infile"] = ""
df["is_leaf"] = ~df.child.isin(df.parent) # setting nodes that are never parents as leaves

# Writing a Makefile 
f = open(path_to_make, "w")
print(f"all: {' '.join(df.outfile.to_list())}\n", file=f)
print("clean:", file=f)
print(f"\t-rm {' '.join(df.outfile.to_list())}\n", file=f)
for i, row in df.iterrows():
    print(f"{row.outfile}: {row.infile} {path_to_slimscript}", file=f)
    print(f"\tslim -s 123 -d \"infile='{row.infile}'\" -d popsize={row.popsize} "
          f"-d \"popname=\'{row.child}\'\" "
          f"-d num_gens={row.edgelen} " f"-d \"outfile='{row.child}.trees'\" "
          f"{path_to_slimscript}\n",
          file=f)
f.close()

We can verify the result is very similar to what we have in the paper, although takes a different strategy
(it does not include merging):

In [5]:
! cat parallel_sims.make

all: root.trees C.trees I.trees B.trees A.trees

clean:
	-rm root.trees C.trees I.trees B.trees A.trees

root.trees:  simulate_branch.slim
	slim -s 123 -d "infile=''" -d popsize=500 -d "popname='root'" -d num_gens=2000 -d "outfile='root.trees'" simulate_branch.slim

C.trees: root.trees simulate_branch.slim
	slim -s 123 -d "infile='root.trees'" -d popsize=50 -d "popname='C'" -d num_gens=250 -d "outfile='C.trees'" simulate_branch.slim

I.trees: root.trees simulate_branch.slim
	slim -s 123 -d "infile='root.trees'" -d popsize=100 -d "popname='I'" -d num_gens=200 -d "outfile='I.trees'" simulate_branch.slim

B.trees: I.trees simulate_branch.slim
	slim -s 123 -d "infile='I.trees'" -d popsize=70 -d "popname='B'" -d num_gens=50 -d "outfile='B.trees'" simulate_branch.slim

A.trees: I.trees simulate_branch.slim
	slim -s 123 -d "infile='I.trees'" -d popsize=40 -d "popname='A'" -d num_gens=50 -d "outfile='A.trees'" simulate_branch.slim



### Running the simulations in parallel

Now, `make` will magically run things in parallel if possible:

In [6]:
%%bash
make -f parallel_sims.make clean
make -f parallel_sims.make -j 3

rm root.trees C.trees I.trees B.trees A.trees
slim -s 123 -d "infile=''" -d popsize=500 -d "popname='root'" -d num_gens=2000 -d "outfile='root.trees'" simulate_branch.slim
// Initial random seed:
123

// RunInitializeCallbacks():
initializeSLiMModelType(modelType = 'WF');
initializeTreeSeq();
initializeMutationRate(1e-08);
initializeMutationType(1, 0.5, "f", -0.01);
initializeGenomicElementType(1, m1, 0.1);
initializeGenomicElement(g1, 0, 999999);
initializeRecombinationRate(1e-09);

// Starting run at tick <start>:
1 

slim -s 123 -d "infile='root.trees'" -d popsize=50 -d "popname='C'" -d num_gens=250 -d "outfile='C.trees'" simulate_branch.slim
slim -s 123 -d "infile='root.trees'" -d popsize=100 -d "popname='I'" -d num_gens=200 -d "outfile='I.trees'" simulate_branch.slim
// Initial random seed:
123

// RunInitializeCallbacks():
initializeSLiMModelType(modelType = 'WF');
initializeTreeSeq();
initializeMutationRate(1e-08);
initializeMutationType(1, 0.5, "f", -0.01);
initializeGenomicEle

In [7]:
! ls

A.trees		I.trees		     phylo.pdf		   union.py
B.trees		parallelizing.ipynb  phylo.tsv
create_make.py	parallel_sims.make   root.trees
C.trees		phylo.csv	     simulate_branch.slim


## Putting it all together: unioning the tree sequences

Next, we put them back together.

In [8]:
def match_nodes(other, ts, split_time):
    node_mapping = np.full(other.num_nodes, -1, dtype='int')
    ts_ids = np.array([n.metadata["slim_id"] for n in ts.nodes()])
    other_ids = np.array([n.metadata["slim_id"] for n in other.nodes()])
    alive_before_split1 = (other.tables.nodes.time >= split_time)
    shared_id = np.isin(other_ids, ts_ids)
    both = np.logical_and(alive_before_split1, shared_id)
    sorted_ids = np.argsort(ts_ids)
    matches = np.searchsorted(
        ts_ids,
        other_ids[both],
        side='left',
        sorter=sorted_ids
    )
    node_mapping[both] = sorted_ids[matches]
    return node_mapping

In [9]:
def union_children(parent, df, merged):
    print(f"Going in: {parent}")
    child_rows = df[df.parent == parent]
    assert (len(child_rows) == 2) or (len(childs) == 0)
    if len(child_rows) == 2:
        children = [row.child for _, row in child_rows.iterrows()]
        for child in children:
            if child not in merged:
                union_children(child, df, merged)
        split_time = merged[children[0]]["depth"]
        assert split_time == merged[children[1]]["depth"] # ultrametric
        print(f'Unioning: {children}, Split time: {split_time}')
        ts0 = merged[children[0]]["ts"]
        ts1 = merged[children[1]]["ts"]
        node_map = match_nodes(ts1, ts0, split_time)
        tsu = ts0.union(ts1, node_map, check_shared_equality=True)
        # the time from tip to start of simulation is split_time plus the
        # length of the edge
        parent_edgelength = df[df.child==parent].edgelen.item()
        merged[parent] = {
            "ts": tsu,
            "depth": split_time + parent_edgelength,
            "children": merged[children[0]]["children"] + merged[children[1]]["children"]
        }

In [10]:
pops = ['A', 'B', 'C']
A, B, C = (tskit.load(f"{x}.trees") for x in pops)
nm = match_nodes(B, A, split_time=50)
AB = A.union(B, nm)
nm = match_nodes(C, AB, split_time=200)
ABC = AB.union(C, nm)

In [11]:
alive = np.where(np.isclose(ABC.tables.nodes.time, 0))[0]
pop_ids = {}
for pop in ABC.populations():
    if pop.metadata is not None:
        pop_ids[pop.metadata['name']] = pop.id

for name in pops:
    pop_samples = ABC.samples(pop_ids[name])
    n_samples = sum(np.isin(pop_samples, alive)) // 2
    print(f"Union-ed tree sequence has {n_samples} samples in population {name},\n"
          f"\tand we specified {df[df.child==name].popsize.item()} individuals in our simulations.")
    assert n_samples == df[df.child==name].popsize.item()

Union-ed tree sequence has 40 samples in population A,
	and we specified 40 individuals in our simulations.
Union-ed tree sequence has 70 samples in population B,
	and we specified 70 individuals in our simulations.
Union-ed tree sequence has 50 samples in population C,
	and we specified 50 individuals in our simulations.


The final result is `ABC`:

In [12]:
ABC